**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *
from scripts.postprocessing import *

[00:58:28] Initializing Normalizer
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/matplotlib/__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/setuptools/_distutils/version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/sklearn_extra/cluster/_commonnn.py:18: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(sklearn.__version__) < LooseVersion("0.23.0"):
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tq

In [3]:
software = Path('/home/tony/DockM8/software')
receptors = [Path('/home/tony/DockM8/testing_ensemble_docking/protein1.pdb'), Path('/home/tony/DockM8/testing_ensemble_docking/protein2.pdb')]
ref_files = [Path('/home/tony/DockM8/testing_ensemble_docking/ref1.sdf'), Path('/home/tony/DockM8/testing_ensemble_docking/ref2.sdf')]
docking_library = Path('/home/tony/DockM8/testing_ensemble_docking/library.sdf')
docking_programs = ['PLANTS']
clustering_metric = 'bestpose_PLANTS'
rescoring_functions = ['GNINA_Affinity','CNN-Score','CNN-Affinity', 'AD4', 'CHEMPLP', 'RFScoreVS']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'GypsumDL'
ncpus = 5
pocket = 'reference'
consensus_method = 'method6'

print('DockM8 is running in ensemble mode...')
        
receptor_dict = {}
for i, receptor in enumerate(receptors):
    receptor_dict[receptor] = ref_files[i]
    
for receptor, ref_file in receptor_dict.items():

    w_dir = Path(receptor).parent / Path(receptor).stem
    print('The working directory has been set to:', w_dir)
    (w_dir).mkdir(exist_ok=True)
    
    #if os.path.isfile(str(receptor).replace('.pdb', '_pocket.pdb')) == False:
    if pocket == 'reference':
        pocket_definition = get_pocket(ref_file, receptor, 8)
        print(pocket_definition)
    if pocket == 'RoG':
        pocket_definition = get_pocket_RoG(ref_file, receptor)
        print(pocket_definition)
    elif pocket == 'dogsitescorer':
        pocket_definition = binding_site_coordinates_dogsitescorer(receptor, w_dir, method='volume')
        print(pocket_definition)
    #else:
        #pocket_definition = calculate_pocket_coordinates_from_pocket_pdb_file((str(receptor).replace('.pdb', '_pocket.pdb')))
        
    if (w_dir / 'final_library.sdf').is_file() == False:
        prepare_library(docking_library, w_dir, id_column, protonation, software, ncpus)
        
    docking(w_dir, receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5)
    concat_all_poses(w_dir, docking_programs)

    print('Loading all poses SDF file...')
    tic = time.perf_counter()
    all_poses = PandasTools.LoadSDF(str(w_dir / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
    print(f'Loaded {len(all_poses)} poses.')
    toc = time.perf_counter()
    print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

    if os.path.isfile(w_dir / 'clustering/'f'{clustering_metric}_clustered.sdf') == False:
        cluster_pebble(clustering_metric, 'KMedoids', w_dir, receptor, all_poses, ncpus)
        
    rescore_all(w_dir, receptor, pocket_definition, software, w_dir / 'clustering' / f'{clustering_metric}_clustered.sdf', rescoring_functions , ncpus)

    apply_consensus_methods(w_dir, clustering_metric, consensus_method, rescoring_functions, 'min_max')
    
ensemble_results = ensemble_consensus(receptors, clustering_metric, consensus_method, 20)

print(ensemble_results)

DockM8 is running in ensemble mode...
The working directory has been set to: /home/tony/DockM8/testing_ensemble_docking/protein1

[2023-Oct-25 01:00:30]: Extracting pocket from /home/tony/DockM8/testing_ensemble_docking/protein1.pdb using /home/tony/DockM8/testing_ensemble_docking/ref1.sdf as reference ligand


/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/biopandas/pdb/pandas_pdb.py:631: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if pd_version < LooseVersion("0.17.0"):
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/biopandas/pdb/pandas_pdb.py:641: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  elif pd_version < LooseVersion("0.23.0"):



[2023-Oct-25 01:00:46]: Finished extracting pocket from /home/tony/DockM8/testing_ensemble_docking/protein1.pdb using /home/tony/DockM8/testing_ensemble_docking/ref1.sdf as reference ligand
{'center': [16.7, -2.69, 17.61], 'size': [16.0, 16.0, 16.0]}
Splitting SDF file final_library.sdf ...


Splitting files: 100%|██████████| 10/10 [00:00<00:00, 1190.14it/s]


Split docking library into 10 files each containing 1 compounds

[2023-Oct-25 01:00:46]: All poses succesfully combined!
Loading all poses SDF file...
Loaded 100 poses.
Finished loading all poses SDF in 0.0594!...

[2023-Oct-25 01:00:46]: *Calculating b metrics and clustering*


Running b clustering...: 100%|██████████| 10/10 [00:00<00:00, 321.88jobs/s]


Invalid metric 'b'
Invalid metric 'b'
Invalid metric 'b'
Invalid metric 'b'
Invalid metric 'b'
Invalid metric 'b'
Invalid metric 'b'
Invalid metric 'b'
Invalid metric 'b'
Invalid metric 'b'


ValueError: No objects to concatenate